In [8]:

import _init_paths

import pickle
import json
from tqdm import tqdm
import torch
import xml.etree.ElementTree as ET
import os
import numpy as np


from model.faster_rcnn.vgg16 import vgg16
from model.utils.config import cfg, cfg_from_list
#from datasets.factory import get_imdb


In [26]:
print(len(data))

17952


In [27]:
data[0]

{'boxes': array([[712, 143, 810, 307]], dtype=uint16),
 'gt_classes': array([2], dtype=int32),
 'gt_ishard': array([0], dtype=int32),
 'gt_overlaps': <1x3 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'flipped': False,
 'seg_areas': array([16335.], dtype=float32),
 'img_id': 0,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/000000.png',
 'width': 1224,
 'height': 370,
 'max_classes': array([2]),
 'max_overlaps': array([1.], dtype=float32)}

In [ ]:
for datainfo in data:
    wh_pair =[(1242,375),(1224,370),(1238,374),(1241,376)]

    width = datainfo['width']
    height = datainfo['height']
    temp = (width,height)
    if (temp not in wh_pair):
        print(datainfo['image'])
        print (width)
        print (height)

# WK value

In [2]:
import pickle
testimg_pickle_path = "data/pickle/BDD_val_1000.pkl"
with open(testimg_pickle_path, 'rb') as handle:
    test_images = pickle.load(handle)

In [6]:
imdb_classes =  ('__background__', 'car', 'person','rider', 'train', 'truck')

In [23]:
def load_model(imdb_classes,model_path, lr):
    
    fasterRCNN = vgg16(imdb_classes, pretrained=False, class_agnostic=False)
    fasterRCNN.create_architecture()
    fasterRCNN.cuda()
    
    
    checkpoint = torch.load(model_path)
    
    
    fasterRCNN.load_state_dict(checkpoint['model'])
    
    params = []
    DOUBLE_BIAS = True
    WEIGHT_DECAY = 0.0005
    BIAS_DECAY = False

    for key, value in dict(fasterRCNN.named_parameters()).items():
        if value.requires_grad:
            if 'bias' in key:
                params += [{'params':[value],'lr':lr*(DOUBLE_BIAS + 1), \
                    'weight_decay': BIAS_DECAY and WEIGHT_DECAY or 0}]
            else:
                params += [{'params':[value],'lr':lr, 'weight_decay': WEIGHT_DECAY}]
    
    optimizer = torch.optim.SGD(params, momentum=0.9) 
    optimizer.load_state_dict(checkpoint['optimizer'])
    return fasterRCNN,optimizer

In [24]:


parties=1
model_list=[None] * parties
load_name = "models/vgg16/multi_ck/multi_ck_FedAvg/faster_rcnn_multi_ck_AVG_10.pth"
for i in range(parties):
    model_list[i], optimizer =load_model(imdb_classes, load_name,lr=0.001)

In [32]:
from frcnn_helper import *

def getWeight(test_images,model_list, batch_size, k):
    
    wk_list = []
    for fasterRCNN in model_list:
#         if args.mGPUs:
#             fasterRCNN = fasterRCNN.module
        X = get_features(fasterRCNN, test_images, batch_size)/255.0
        wk_value = within_cluster_dispersion(X, n_cluster=k)
        wk_list.append(wk_value)
        print(wk_value)
    
    return wk_list 

In [33]:
batch_size=16
k=5
wk_list_curr = getWeight(test_images,model_list, batch_size,k)


0.03316984123371074


In [ ]:


# get within class dispersion        


if i==1:
    wk_diff = wk_list_curr
else:
    wk_diff=[]
    for list1_c, list2_p in zip(wk_list_prev, wk_list_curr ):        
        wk_diff.append(list1_c-list2_p)

print('diff={}'.format(wk_diff))

wk_ratio = softmax(wk_diff).tolist()    
print('wk_ratio={}'.format(wk_ratio))

#wk_ratio = [x / sum(wk_diff) for x in wk_diff]
#keep wk to previous
wk_list_prev = wk_list_curr

# BDD converter

In [14]:

json_path = '/work/superorange5/bdd100k/labels/bdd100k_labels_images_train.json'
with open(json_path) as f:
    j = f.read()
    data = json.loads(j)

    

        
#     print(datum['name'])
    
# for datum in tqdm(data):


In [29]:
for datum in data:
    if datum['name'] == '1df7314f-da89fc35.jpg':
        for bb in datum['labels']:
            if bb['category'] == 'car':
                print(bb)

{'category': 'car', 'attributes': {'occluded': False, 'truncated': True, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 732.648216, 'y1': 273.850561, 'x2': 1277.473049, 'y2': 615.293917}, 'id': 324596}
{'category': 'car', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 545.59664, 'y1': 356.984595, 'x2': 679.204909, 'y2': 466.840282}, 'id': 324597}
{'category': 'car', 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 498.804206, 'y1': 382.020696, 'x2': 552.247514, 'y2': 421.113478}, 'id': 324598}
{'category': 'car', 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 485.443379, 'y1': 384.49491, 'x2': 521.072251, 'y2': 409.732027}, 'id': 324599}
{'category':

In [90]:
len(data)

10000

In [91]:
fp = open("val.txt", "w")

for datum in data:
    if(datum['attributes']['timeofday'] =='daytime'):
        fp.write(datum['name'].replace('.jpg','')+"\n" )
fp.close()

In [6]:
data[0].keys()

dict_keys(['name', 'attributes', 'timestamp', 'labels'])

In [ ]:
for datum in data:
   
    # bounding box
    for label in datum['labels']:
        classes = label['category']
        if classes =='motor':
            print("motorcycle")
        elif classes =='bike':
            print("bicycle")
        elif classes =='traffic light':
            print("skip")
        elif classes =='traffic sign':
            print("skip")
        


In [3]:
data[0]['name']   


'0000f77c-6257be58.jpg'

In [8]:
data[0]['attributes']['timeofday']


'daytime'

In [9]:
data[0]['labels'][0]['category']


'traffic light'

## read XML

In [53]:
import xml.etree.ElementTree as ET
import numpy as np

import os
# assign directory
#directory = 'data/KITTI/Annotations/'
directory = 'data/BDD100K/Annotations/'
count=0
    # check exist car or not
for filename in os.listdir(directory):
    #print(filename)
    
    tree = ET.parse(os.path.join(directory,filename))
    root = tree.getroot()

    objs = tree.findall('object')


            # Load object bounding boxes into a data frame.
    num_objs =0
    for ix, obj in enumerate(objs):
        bbox = obj.find('bndbox')

#         x1 = max(float(bbox.find('xmin').text), 0)
#         y1 = max(float(bbox.find('ymin').text), 0)
#         x2 = max(float(bbox.find('xmax').text), 0)
#         y2 = max(float(bbox.find('ymax').text), 0)


        class_name = obj.find('name').text.lower().strip()
        if class_name=='train':            
            num_objs+=1
        #print(class_name)
    #print(num_objs)
    if (num_objs)==0:
        #print(filename)
        count+=1
print(count)
    

79744


# load model

In [185]:
model_path = "models/vgg16/multi_ck/faster_rcnn_1_20_17897.pth"
model  = torch.load(model_path)

In [188]:
classes = np.asarray(['__background__', 'car', 'person', 'bus', 'bicycle', 'motorcycle','rider', 'train', 'truck'])
set_cfgs = ['ANCHOR_SCALES', '[8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]', 'MAX_NUM_GT_BOXES', '20'] 

cfg_from_list(set_cfgs)
fasterRCNN = vgg16(classes, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
fasterRCNN.eval()
fasterRCNN.cuda()

vgg16(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 18, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 36, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer()
    (RPN_anchor_target): _AnchorTargetLayer()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): ROIPool(output_size=(7, 7), spatial_scale=0.0625)
  (RCNN_roi_align): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0)
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), 

In [189]:
fasterRCNN.load_state_dict(model['model'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## check xml class

In [5]:

def read_annotation(x):
    filename = os.path.join('data/BDD100K','Annotations',x +'.xml')
    #print(filename)
    tree = ET.parse(filename)
    objs = tree.findall('object')

    for ix, obj in enumerate(objs):
        bbox = obj.find('bndbox')
        cls=obj.find('name').text.lower().strip()
        if(cls=='train'):
            print(filename)
        if('motorbike' == cls):
            continue
        #cls = _class_to_ind[obj.find('name').text.lower().strip()]
    return filename

#image_set_file = os.path.join('data/SIM10K/VOC2012', 'ImageSets', 'Main',  'trainval10k.txt')
image_set_file = os.path.join('data/BDD100K', 'ImageSets', 'Main',  'train.txt')
classes = ('__background__', 'car', 'person', 'bus', 'bicycle', 'motorcycle','rider', 'train', 'truck')
num_classes = len(classes)
_class_to_ind = dict(zip(classes, range(num_classes)))
with open(image_set_file) as f:
    for x in f.readlines():
        x = x.rstrip("\n")
        read_annotation(x)

data/BDD100K/Annotations/002d290d-89f4e5c0.xml
data/BDD100K/Annotations/02f95a54-6494531c.xml
data/BDD100K/Annotations/0406b065-a759da9c.xml
data/BDD100K/Annotations/04488248-af1d57c1.xml
data/BDD100K/Annotations/06cdf0b1-6fec3924.xml
data/BDD100K/Annotations/0710389f-038050ae.xml
data/BDD100K/Annotations/0710389f-038050ae.xml
data/BDD100K/Annotations/0710389f-038050ae.xml
data/BDD100K/Annotations/0710389f-038050ae.xml
data/BDD100K/Annotations/0c1d07e3-8cb61d13.xml
data/BDD100K/Annotations/0c1d07e3-8cb61d13.xml
data/BDD100K/Annotations/0ed0d326-5ec5573f.xml
data/BDD100K/Annotations/1125356f-edb7f7c8.xml
data/BDD100K/Annotations/13250071-2043b0ff.xml
data/BDD100K/Annotations/148b9b8a-7d0771aa.xml
data/BDD100K/Annotations/152b261b-776bfbb0.xml
data/BDD100K/Annotations/1be335cc-3074a07c.xml
data/BDD100K/Annotations/1c8b6894-dca9308f.xml
data/BDD100K/Annotations/1df7314f-da89fc35.xml
data/BDD100K/Annotations/1df7314f-da89fc35.xml
data/BDD100K/Annotations/2124b66a-da9e92f8.xml
data/BDD100K/

## voc_eval

In [47]:
import pickle
data_cache_path = 'data/cache'
pkl_file = os.path.join(data_cache_path, 'kitti_train' + '_gt_roidb.pkl')

In [48]:
with open(pkl_file, 'rb') as f:
    data = pickle.load(f)

In [49]:
len(data)

11968

In [52]:
data[3180]

{'boxes': array([[829, 161, 888, 286],
        [758, 178, 795, 271]], dtype=uint16),
 'gt_classes': array([2, 2], dtype=int32),
 'gt_ishard': array([0, 0], dtype=int32),
 'gt_overlaps': <2x6 sparse matrix of type '<class 'numpy.float32'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 'flipped': False,
 'seg_areas': array([7560., 3572.], dtype=float32),
 'img_id': 3180,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/003955.png',
 'width': 1242,
 'height': 375,
 'max_classes': array([2, 2]),
 'max_overlaps': array([1., 1.], dtype=float32)}

In [3]:
devkit_path = '/work/superorange5/bdd100k'
cachedir = os.path.join(devkit_path, 'annotations_cache')
image_set='val'
imagesetfile = os.path.join(devkit_path,  'ImageSets',
            'Main',   image_set + '.txt')
cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)


In [4]:
annopath = os.path.join(devkit_path, 'Annotations', '{:s}.xml')

In [5]:
with open(imagesetfile, 'r') as f:
    lines = f.readlines()
imagenames = [x.strip() for x in lines]

In [6]:
cachefile

'/work/superorange5/bdd100k/ImageSets/Main/val.txt_annots.pkl'

In [7]:
len(imagenames)

10000

In [15]:
import lib.datasets.voc_eval as voc_eval

## generate gt pkl

In [29]:
recs = {}
#for i in range(1000):
#    imagename = imagenames[i]
for i, imagename in enumerate(imagenames):
    recs[imagename] = voc_eval.parse_rec(annopath.format(imagename))
    if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
print('Saving cached annotations to {:s}'.format(cachefile))
with open(cachefile, 'wb') as f:
        pickle.dump(recs, f)

Saving cached annotations to /work/superorange5/bdd100k/ImageSets/Main/val.txt_annots.pkl


In [30]:
recs['b1ca8418-84a133a0']

[{'name': 'traffic light', 'truncated': 0, 'bbox': [449, 123, 471, 146]},
 {'name': 'traffic sign', 'truncated': 0, 'bbox': [638, 206, 653, 231]},
 {'name': 'traffic sign', 'truncated': 0, 'bbox': [199, 179, 231, 212]},
 {'name': 'car', 'truncated': 0, 'bbox': [439, 222, 698, 430]},
 {'name': 'car', 'truncated': 1, 'bbox': [1136, 200, 1278, 333]},
 {'name': 'motor', 'truncated': 1, 'bbox': [1204, 232, 1277, 330]}]

## extract gt

In [74]:

classname = 'car' 
class_recs = {}
npos = 0
ovthresh=0.5
for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    #difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos +  len(R) #sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
     #                        'difficult': difficult,
                             'det': det}


## read dets

In [75]:
detfile = os.path.join(devkit_path,'results/Main/comp4_det_val_'+classname+'.txt')
with open(detfile, 'r') as f:
    lines = f.readlines()

splitlines = [x.strip().split(' ') for x in lines]
image_ids = [x[0] for x in splitlines]
confidence = np.array([float(x[1]) for x in splitlines])
BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

nd = len(image_ids)
tp = np.zeros(nd)
fp = np.zeros(nd)

if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d, :].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
        # compute overlaps
        # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
        #if not R['difficult'][jmax]:
            if not R['det'][jmax]:
                tp[d] = 1.
                R['det'][jmax] = 1
            else:
                fp[d] = 1.
        else:
            fp[d] = 1.

  # compute precision recall
fp = np.cumsum(fp)
tp = np.cumsum(tp)
rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
ap = voc_eval.voc_ap(rec, prec, False)

In [76]:
ap

0.2416113877975409

In [36]:
if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 10 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    print('Saving cached annotations to {:s}'.format(cachefile))
    with open(cachefile, 'wb') as f:
      pickle.dump(recs, f)

In [26]:

  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    print('Saving cached annotations to {:s}'.format(cachefile))
    with open(cachefile, 'wb') as f:
      pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    #difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    #npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
     #                        'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      R = class_recs[image_ids[d]]
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        #if not R['difficult'][jmax]:
        if not R['det'][jmax]:
          tp[d] = 1.
          R['det'][jmax] = 1
        else:
          fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

{'boxes': array([[1001,  282, 1041,  327],
        [ 215,  172,  275,  230],
        [ 797,  313,  830,  342],
        [ 653,  303,  685,  316],
        [ 707,  312,  716,  328],
        [ 626,  296,  636,  317],
        [ 317,  289,  329,  307],
        [ 271,  288,  309,  302],
        [ 221,  301,  232,  312],
        [ 206,  338,  282,  388],
        [  47,  344,  130,  401],
        [ 247,  344,  347,  396],
        [   0,  337,   52,  403],
        [ 650,  354,  664,  382],
        [ 649,  367,  664,  388],
        [ 712,  337,  726,  353],
        [ 684,  357,  721,  393],
        [ 706,  364,  734,  391],
        [ 727,  366,  761,  401],
        [ 749,  362,  809,  409],
        [ 787,  358,  906,  425],
        [ 880,  376,  960,  452],
        [ 936,  336, 1212,  484],
        [1204,  416, 1279,  530],
        [ 603,  341,  610,  353],
        [ 591,  328,  598,  342],
        [ 529,  332,  537,  342],
        [ 552,  356,  565,  368],
        [ 574,  352,  586,  369],
     

## bg_num_rois = 0 and fg_num_rois = 0, this should not happen!

In [84]:

pkl_filepath = 'data/cache/multi_ck_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    multick_data = pickle.load(f)

In [87]:
len(multick_data)

17898

In [85]:
pkl_filepath = 'data/cache/kitti_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    kiti_data = pickle.load(f)

In [86]:
len(kiti_data)

17952

In [84]:
kiti_data[40]

{'boxes': array([[414, 143, 512, 307]], dtype=uint16),
 'gt_overlaps': <1x3 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'gt_classes': array([2], dtype=int32),
 'flipped': True}

In [111]:
multick_data[11914]

{'boxes': array([[414, 143, 512, 307]], dtype=uint16),
 'gt_overlaps': <1x9 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'gt_classes': array([2], dtype=int32),
 'flipped': True,
 'img_id': 11914,
 'image': '/home/superorange5/Research/FedWCD/data/KITTI/JPEGImages/000000.png',
 'width': 1224,
 'height': 370,
 'max_classes': array([2]),
 'max_overlaps': array([1.], dtype=float32)}

## roidb rank width error

In [18]:
data_cache_path = 'data/cache'

In [25]:

pkl_filepath = 'data/cache/kitti_train_gt_roidb.pkl'
with open(pkl_filepath, 'rb') as f:
    data = pickle.load(f)

In [62]:
for s in 'kitti_train'.split('+'):
    print(s)

kitti_train


### rank

In [63]:
pkl_file = os.path.join(data_cache_path, 'kitti_train_small_gt_roidb.pkl')

with open(pkl_file, 'rb') as f:
    roidb = pickle.load(f)

#roidb = filter_roidb(roidb)
train_size = len(roidb)

In [64]:
train_size

40

In [58]:
roidb[11967]

IndexError: list index out of range

In [45]:
roidb[11968]

{'boxes': array([[414, 143, 512, 307]], dtype=uint16),
 'gt_overlaps': <1x3 sparse matrix of type '<class 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'gt_classes': array([2], dtype=int32),
 'flipped': True}

In [54]:


ratio_large = 2 # largest ratio to preserve.
ratio_small = 0.5 # smallest ratio to preserve.    
    
ratio_list = []
 #   print(len(roidb))
for i in range(len(roidb)):
    print('#####'+str(i))
      
    width = roidb[i]['width']
    height = roidb[i]['height']


#####0
#####1
#####2
#####3
#####4
#####5
#####6
#####7
#####8
#####9
#####10
#####11
#####12
#####13
#####14
#####15
#####16
#####17
#####18
#####19
#####20
#####21
#####22
#####23
#####24
#####25
#####26
#####27
#####28
#####29
#####30
#####31
#####32
#####33
#####34
#####35
#####36
#####37
#####38
#####39


In [49]:
def combined_roidb(imdb_names, training=True):
  """
  Combine multiple roidbs
  """

  def get_training_roidb(imdb):
    """Returns a roidb (Region of Interest database) for use in training."""
    if cfg.TRAIN.USE_FLIPPED:
      print('Appending horizontally-flipped training examples...')
      imdb.append_flipped_images()
      print('done')

    print('Preparing training data...')

    prepare_roidb(imdb)
    #ratio_index = rank_roidb_ratio(imdb)
    print('done')

    return imdb.roidb
  
  def get_roidb(imdb_name):
    imdb = get_imdb(imdb_name)
    print('Loaded dataset `{:s}` for training'.format(imdb.name))
    imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
    print('Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD))
    roidb = get_training_roidb(imdb)
    return roidb

  roidbs = [get_roidb(s) for s in imdb_names.split('+')]
  roidb = roidbs[0]
  if len(roidbs) > 1:
    for r in roidbs[1:]:
      roidb.extend(r)
    tmp = get_imdb(imdb_names.split('+')[1])
    imdb = datasets.imdb.imdb(imdb_names, tmp.classes)
  else:
    imdb = get_imdb(imdb_names)

  if training:
    roidb = filter_roidb(roidb)


In [126]:
for i in range(len(kiti_data)):
  #    print('#####'+str(i))
    width = multick_data[i]['width']
    #print(width)

## create a new

In [119]:
new_multick = multick_data[0:100]+multick_data[2965:3065]

In [120]:
len(new_multick)

200

In [121]:
cache_file = 'data/cache/multi_ck_trainsmall'
with open(cache_file, 'wb') as fid:
    pickle.dump(new_multick, fid)

In [ ]:
for data_info in data:
    bboxes = data_info['boxes']
    if len(bboxes) <2:
        print (data_info)